### Oracle Vector DB wrapped as a llama-index custom Vector Store

* ispired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html

In [1]:
import logging
import sys

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document

import oci
import ads
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
import oracledb

from config_private import COMPARTMENT_OCID, ENDPOINT

from oci_utils import load_oci_config
from oracle_vector_db import OracleVectorStore

In [2]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649


In [3]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual
MODEL_NAME = "cohere.embed-english-v3.0"

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=MODEL_NAME,
    auth=ads.auth.api_keys(oci_config),
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={
        "service_endpoint": ENDPOINT
    },
)

#### Using the wrapper for the DB Vector Store

In [5]:
v_store = OracleVectorStore(verbose=False)

In [6]:
question = "What is JSON Relational Duality? Explain with details"

In [7]:
# embed the query using OCI GenAI
query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

#### Use our Vector Store DB

In [8]:
%%time

q_result = v_store.query(query_obj)

CPU times: user 21.6 ms, sys: 4.41 ms, total: 26 ms
Wall time: 658 ms


In [9]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Dod. id: {id}")
    print(f"Similarity: {-sim}")
    print(n.text)
    print("")

Dod. id: efcd1add-f3c9-4189-b10d-356639f6e4df
Similarity: 0.487
2 Application Development JSON JSON-Relational Duality JSON Relational Duality Views are fully updatable JSON views over relational data. Data is still stored in relational tables in a highly efficient normalized format but can be accessed by applications in the form of JSON documents. Duality views provide you with game-changing flexibility and simplicity by overcoming the historical challenges developers have faced when building applications using relational or document models. Related Resources View Documentation JSON Schema JSON Schema-based validation is allowed with the SQL condition IS JSON and with a PL/SQL utility function. A JSON schema is a JSON document that specifies allowed properties (field names) and the corresponding allowed data types, and whether they are optional or mandatory. By default, JSON data is schemaless, providing flexibility. However, you may want to ensure that your JSON data contains particu

#### Integrate in the bigger RAG picture

In [10]:
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={
        "service_endpoint": ENDPOINT
    },
)

In [11]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [12]:
index = VectorStoreIndex.from_vector_store(vector_store=v_store,
    service_context=service_context
)

In [13]:
query_engine = index.as_query_engine(similarity_top_k=5)

In [14]:
%%time

response = query_engine.query(question)

print(f"Question: {question}") 
print(response.response)
print("")

Question: What is JSON Relational Duality? Explain with details
JSON Relational Duality is a concept in Oracle Database that allows for the creation of fully updatable JSON views over relational data. It provides a game-changing flexibility and simplicity for developers who want to work with both relational and document models in their applications.

With JSON Relational Duality, data is still stored in highly efficient, normalized relational tables. However, applications can access this data in the form of JSON documents, which have become a popular and prevalent format for unstructured data. This is especially useful for scenarios where you need to work with simple, structured data, semistructured data, or even complex structured data.

To illustrate how this works, consider the following example:

You have a relational table that stores information about customers, including their names, addresses, and other details. You can use JSON Relational Duality to create a JSON view that exp

#### Additional tests

In [23]:
# using the Oracle Vectre Store get embeddings from id
arr = v_store.get(text_id="efcd1add-f3c9-4189-b10d-356639f6e4df")

import numpy as np

np.array(arr)

array([ 0.01416779, -0.01168823, -0.07788086, ...,  0.03286743,
       -0.0239563 ,  0.02148438])